In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 🧾 Fine-Tune Invoice Parser\n",
    "This Colab notebook fine-tunes **SmolVLM / Idefics-3** on the dataset `mychen76/invoices-and-receipts_ocr_v1` using **TRL + LoRA**.\n",
    "\n",
    "It includes: data loading, preprocessing, fine-tuning, and evaluation — all in one file."
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "!pip install -U \"transformers>=4.43.0\" \"trl>=0.9.0\" \"peft>=0.11\" \"datasets\" \"torch\" \"bitsandbytes\" \"accelerate\""
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "from datasets import load_dataset\n",
    "import json\n",
    "from transformers import Idefics3ForConditionalGeneration, AutoProcessor\n",
    "import torch\n",
    "from peft import LoraConfig, get_peft_model\n",
    "from trl import SFTTrainer, SFTConfig\n",
    "import os\n",
    "\n",
    "os.environ['WANDB_DISABLED'] = 'true'\n",
    "\n",
    "ds = load_dataset('mychen76/invoices-and-receipts_ocr_v1')\n",
    "\n",
    "def flatten_example(example):\n",
    "    parsed = json.loads(example['parsed_data'])\n",
    "    structured = parsed.get('json', '{}')\n",
    "    try:\n",
    "        structured_json = json.loads(structured.replace(\"'\", '\"'))\n",
    "    except:\n",
    "        structured_json = {'error': 'invalid_json'}\n",
    "    prompt = 'Extract all invoice fields and return as JSON.'\n",
    "    output = json.dumps(structured_json)\n",
    "    return {'text': f'{prompt}\\n{output}'}\n",
    "\n",
    "flat_train = ds['train'].map(flatten_example)\n",
    "flat_valid = ds['valid'].map(flatten_example)\n",
    "\n",
    "model = Idefics3ForConditionalGeneration.from_pretrained(\n",
    "    'HuggingFaceTB/SmolVLM-Instruct',\n",
    "    torch_dtype=torch.float16,\n",
    "    device_map='auto'\n",
    ")\n",
    "processor = AutoProcessor.from_pretrained('HuggingFaceTB/SmolVLM-Instruct')\n",
    "\n",
    "lora_config = LoraConfig(r=8, lora_alpha=32, lora_dropout=0.1, target_modules=['q_proj', 'v_proj'], bias='none', task_type='CAUSAL_LM')\n",
    "model = get_peft_model(model, lora_config)\n",
    "\n",
    "sft_config = SFTConfig(per_device_train_batch_size=2, num_train_epochs=3, learning_rate=1e-4, fp16=True, output_dir='./outputs', report_to='none')\n",
    "trainer = SFTTrainer(model=model, args=sft_config, train_dataset=flat_train, eval_dataset=flat_valid, processing_class=processor)\n",
    "trainer.train()\n",
    "trainer.model.save_pretrained('./fine_tuned_model')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔍 Evaluation"
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "from transformers import Idefics3ForConditionalGeneration, AutoProcessor\n",
    "from datasets import load_dataset\n",
    "import torch, json\n",
    "\n",
    "model = Idefics3ForConditionalGeneration.from_pretrained('./fine_tuned_model', torch_dtype='auto', device_map='auto')\n",
    "processor = AutoProcessor.from_pretrained('./fine_tuned_model')\n",
    "ds = load_dataset('mychen76/invoices-and-receipts_ocr_v1')['valid']\n",
    "\n",
    "for i, ex in enumerate(ds.select(range(5))):\n",
    "    parsed = json.loads(ex['parsed_data']).get('json', '{}')\n",
    "    try:\n",
    "        target_json = json.loads(parsed.replace(\"'\", '\"'))\n",
    "    except:\n",
    "        target_json = {'error': 'invalid_json'}\n",
    "    prompt = 'Extract all invoice fields and return as JSON.'\n",
    "    target_text = json.dumps(target_json)\n",
    "    full = f'{prompt}\\n{target_text}'\n",
    "    enc = processor.tokenizer(full, return_tensors='pt', truncation=True, padding='max_length', max_length=512)\n",
    "    labels = enc['input_ids'].clone()\n",
    "    prompt_len = len(processor.tokenizer(prompt)['input_ids'])\n",
    "    labels[:, :prompt_len] = -100\n",
    "    enc = {k: v.to(model.device) for k, v in enc.items()}\n",
    "    labels = labels.to(model.device)\n",
    "    with torch.no_grad():\n",
    "        loss = model(**enc, labels=labels).loss\n",
    "    print(f'[{i}] Loss: {loss.item():.4f}')\n",
    "    out = processor.tokenizer.batch_decode(model.generate(**enc, max_new_tokens=256), skip_special_tokens=True)[0]\n",
    "    print('Prediction:', out[:300])"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}
